In [1]:
import pygame
import sys
import math
import random
import os
from pygame import gfxdraw

# Initialize Pygame
pygame.init()
pygame.mixer.init()

# Screen dimensions
WIDTH, HEIGHT = 1000, 700
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("The Ballad of Casey Jones - Karaoke")

# Colors
BACKGROUND = (10, 15, 25)
TEXT_COLOR = (220, 220, 220)
HIGHLIGHT_COLOR = (255, 215, 0)
SUBDUED_COLOR = (120, 120, 140)
RAIL_COLOR = (150, 100, 50)
TRAIN_COLOR = (80, 80, 100)
SMOKE_COLOR = (180, 180, 200, 180)
FIRE_COLOR = (255, 100, 0, 200)

# Fonts
title_font = pygame.font.SysFont("Arial", 48, bold=True)
lyric_font = pygame.font.SysFont("Arial", 32)
subtitle_font = pygame.font.SysFont("Arial", 28)
info_font = pygame.font.SysFont("Arial", 20)

# Lyrics with timing information (in seconds)
aligned_lyrics = [
    (6.483, 10.55, "The Workers on the S. P. line to strike sent out a call;"),
    (10.6, 15.555, "But Casey Jones, the engineer, he wouldn't strike at all;"),
    (15.605, 20.55, "His boiler it was leaking, and its drivers on the bum,"),
    (20.6, 26.5, "And his engine and its bearings, they were all out of plumb."),
    (27.5, 31.555, "Casey Jones kept his junk pile running;"),
    (31.605, 35.55, "Casey Jones was working double time;"),
    (35.6, 39.55, "Casey Jones got a wooden medal,"),
    (39.6, 44.5, "For being good and faithful on the S. P. line."),
    (45.644, 50.55, "The workers said to Casey: \"Won't you help us win this strike?\""),
    (50.6, 55.55, "But Casey said: \"Let me alone, you'd better take a hike.\""),
    (55.6, 60.55, "Then some one put a bunch of railroad ties across the track,"),
    (60.6, 66.499, "And Casey hit the river bottom with an awful crack."),
    (67.5, 72.194, "Casey Jones hit the river bottom;"),
    (72.244, 75.721, "Casey Jones broke his blessed spine;"),
    (75.771, 79.55, "Casey Jones was an Angelino,"),
    (79.6, 84.46, "He took a trip to heaven on the S. P. line."),
    (85.5, 90.55, "When Casey Jones got up to heaven, to the Pearly Gate,"),
    (90.6, 95.55, "He said: \"I'm Casey Jones, the guy that pulled the S. P. freight.\""),
    (95.6, 100.55, "\"You're just the man,\" said Peter, \"our musicians went on strike;\""),
    (100.6, 106.482, "\"You can get a job a-scabbing any time you like.\""),
    (107.505, 111.869, "Casey Jones got up to heaven;"),
    (111.919, 114.21, "Casey Jones was doing mighty fine;"),
]


# Particle system for effects
class Particle:
    def __init__(self, x, y, color, velocity, size, decay=0.98):
        self.x = x
        self.y = y
        self.color = color
        self.vx, self.vy = velocity
        self.size = size
        self.decay = decay
        self.life = 1.0
        
    def update(self):
        self.x += self.vx
        self.y += self.vy
        self.vx *= self.decay
        self.vy *= self.decay
        self.size *= self.decay
        self.life -= 0.02
        
    def draw(self, surface):
        alpha = int(255 * self.life)
        color = (*self.color[:3], alpha)
        pygame.gfxdraw.filled_circle(surface, int(self.x), int(self.y), int(self.size), color)

# Train animation class
class Train:
    def __init__(self):
        self.x = WIDTH + 200
        self.y = HEIGHT - 100
        self.speed = 2
        self.smoke_particles = []
        self.fire_particles = []
        
    def update(self, current_time):
        # Move the train
        self.x -= self.speed
        if self.x < -500:
            self.x = WIDTH + 200
        
        # Create smoke
        if random.random() > 0.3:
            self.smoke_particles.append(
                Particle(
                    self.x + 40, self.y - 30,
                    SMOKE_COLOR,
                    (random.uniform(-0.5, 0.5), random.uniform(-2, -1)),
                    random.uniform(5, 10)
                )
            )
            
        # Create fire at certain times
        if 55 < current_time < 65 or 135 < current_time < 145:
            self.fire_particles.append(
                Particle(
                    self.x + 40, self.y - 5,
                    FIRE_COLOR,
                    (random.uniform(-1, 1), random.uniform(-0.5, 0.5)),
                    random.uniform(2, 5),
                    0.9
                )
            )
        
        # Update particles
        for particle in self.smoke_particles[:]:
            particle.update()
            if particle.life <= 0:
                self.smoke_particles.remove(particle)
                
        for particle in self.fire_particles[:]:
            particle.update()
            if particle.life <= 0:
                self.fire_particles.remove(particle)
        
    def draw(self, surface):
        # Draw smoke
        for particle in self.smoke_particles:
            particle.draw(surface)
            
        # Draw fire
        for particle in self.fire_particles:
            particle.draw(surface)
        
        # Draw train
        pygame.draw.rect(surface, TRAIN_COLOR, (self.x, self.y - 30, 120, 30))
        pygame.draw.rect(surface, (60, 60, 80), (self.x, self.y - 30, 120, 30), 2)
        
        # Draw smokestack
        pygame.draw.rect(surface, TRAIN_COLOR, (self.x + 30, self.y - 60, 20, 30))
        
        # Draw wheels
        for i in range(3):
            pygame.draw.circle(surface, (40, 40, 40), (int(self.x + 30 + i*30), int(self.y)), 12)
            pygame.draw.circle(surface, (20, 20, 20), (int(self.x + 30 + i*30), int(self.y)), 12, 2)
            
        # Draw light
        pygame.draw.circle(surface, (255, 255, 200), (int(self.x + 110), int(self.y - 15)), 8)

# Main function
def main():
    clock = pygame.time.Clock()
    train = Train()
    particles = []
    running = True
    paused = False
    
    # Create stars in background
    stars = []
    for _ in range(100):
        x = random.randint(0, WIDTH)
        y = random.randint(0, HEIGHT)
        size = random.uniform(0.5, 2)
        brightness = random.randint(150, 255)
        stars.append((x, y, size, brightness))
    
    # Load and play the song
    try:
        # Try to load a placeholder song (replace with your actual song file)
        pygame.mixer.music.load("/home/rustom/anaKonda/Lyrics/SpotiDownloader.com - Casey Jones (The Union Scab) - Pete Seeger.mp3")
        print("Loaded placeholder song. Replace with your actual song file.")
    except:
        print("Couldn't load song file. Using simulated timing.")
    
    # Create a simulated song timer
    song_start_time = pygame.time.get_ticks()
    
    # Draw background elements
    def draw_background():
        # Draw gradient background
        for y in range(HEIGHT):
            color = (10, max(5, 15 - y//50), max(10, 25 - y//30))
            pygame.draw.line(screen, color, (0, y), (WIDTH, y))
        
        # Draw stars
        for x, y, size, brightness in stars:
            pygame.draw.circle(screen, (brightness, brightness, brightness), (x, y), size)
        
        # Draw ground
        pygame.draw.rect(screen, (20, 30, 10), (0, HEIGHT - 50, WIDTH, 50))
        
        # Draw rails
        pygame.draw.rect(screen, RAIL_COLOR, (0, HEIGHT - 30, WIDTH, 10))
        for x in range(0, WIDTH, 30):
            pygame.draw.rect(screen, (100, 70, 40), (x, HEIGHT - 35, 15, 5))
        
        # Draw ties
        for x in range(0, WIDTH, 40):
            pygame.draw.rect(screen, (80, 60, 30), (x, HEIGHT - 40, 8, 20))
    
    # Draw lyrics
    def draw_lyrics(current_time):
        # Draw title
        title = title_font.render("The Ballad of Casey Jones", True, HIGHLIGHT_COLOR)
        subtitle = subtitle_font.render("A Labor Song by Joe Hill", True, SUBDUED_COLOR)
        screen.blit(title, (WIDTH//2 - title.get_width()//2, 30))
        screen.blit(subtitle, (WIDTH//2 - subtitle.get_width()//2, 90))
        
        # Draw current and upcoming lyrics
        current_lyric = None
        next_lyrics = []
        
        for start, end, text in aligned_lyrics:
            if start <= current_time <= end:
                current_lyric = (text, start, end)
            elif current_time < start and len(next_lyrics) < 3:
                next_lyrics.append((text, start))
        
        # Draw current lyric
        if current_lyric:
            text, start, end = current_lyric
            elapsed = current_time - start
            duration = end - start
            
            # Calculate progress for word highlighting
            progress = min(1.0, elapsed / duration)
            
            # Render the current lyric with word-by-word highlighting
            words = text.split()
            word_widths = [lyric_font.size(word + " ")[0] for word in words]
            total_width = sum(word_widths) - lyric_font.size(" ")[0]
            
            x_pos = WIDTH // 2 - total_width // 2
            y_pos = HEIGHT // 2 - 20
            
            # Draw each word with appropriate highlighting
            word_progress = progress * len(words)
            for i, word in enumerate(words):
                # Determine color based on progress
                if i < int(word_progress):
                    color = HIGHLIGHT_COLOR
                elif i == int(word_progress):
                    # Partially highlight the current word
                    partial = word_progress - i
                    color = (
                        int(HIGHLIGHT_COLOR[0] * partial + TEXT_COLOR[0] * (1 - partial)),
                        int(HIGHLIGHT_COLOR[1] * partial + TEXT_COLOR[1] * (1 - partial)),
                        int(HIGHLIGHT_COLOR[2] * partial + TEXT_COLOR[2] * (1 - partial))
                    )
                else:
                    color = TEXT_COLOR
                
                word_surface = lyric_font.render(word, True, color)
                screen.blit(word_surface, (x_pos, y_pos))
                x_pos += word_widths[i]
        
        # Draw next lyrics
        y_offset = HEIGHT // 2 + 60
        for i, (text, start) in enumerate(next_lyrics):
            if i == 0:
                color = (180, 180, 200)
            else:
                color = SUBDUED_COLOR
            lyric_surface = lyric_font.render(text, True, color)
            screen.blit(lyric_surface, (WIDTH//2 - lyric_surface.get_width()//2, y_offset))
            y_offset += 45
    
    # Draw progress bar
    def draw_progress_bar(current_time):
        total_duration = aligned_lyrics[-1][1] + 5
        progress = current_time / total_duration
        
        # Draw background bar
        bar_width = WIDTH - 100
        bar_height = 8
        bar_x = 50
        bar_y = HEIGHT - 80
        
        pygame.draw.rect(screen, (40, 40, 60), (bar_x, bar_y, bar_width, bar_height))
        pygame.draw.rect(screen, (80, 80, 100), (bar_x, bar_y, int(bar_width * progress), bar_height))
        pygame.draw.rect(screen, (100, 100, 120), (bar_x, bar_y, bar_width, bar_height), 2)
        
        # Draw time info
        time_text = info_font.render(f"{int(current_time//60)}:{int(current_time%60):02d} / {int(total_duration//60)}:{int(total_duration%60):02d}", True, TEXT_COLOR)
        screen.blit(time_text, (WIDTH//2 - time_text.get_width()//2, HEIGHT - 70))
    
    # Draw controls
    def draw_controls():
        controls = info_font.render("SPACE: Play/Pause  |  R: Restart  |  ESC: Quit", True, SUBDUED_COLOR)
        screen.blit(controls, (WIDTH//2 - controls.get_width()//2, HEIGHT - 30))
    
    # Draw volume controls
    def draw_volume_controls(volume):
        # Draw volume background
        pygame.draw.rect(screen, (40, 40, 60), (20, 20, 150, 30))
        pygame.draw.rect(screen, (100, 100, 120), (20, 20, 150, 30), 2)
        
        # Draw volume level
        pygame.draw.rect(screen, (80, 80, 100), (20, 20, int(150 * volume), 30))
        
        # Draw volume text
        volume_text = info_font.render("Volume: " + str(int(volume * 100)) + "%", True, TEXT_COLOR)
        screen.blit(volume_text, (25, 25))
    
    # Start playing the song
    pygame.mixer.music.play()
    music_playing = True
    
    # Set initial volume
    volume = 0.7
    pygame.mixer.music.set_volume(volume)
    
    # Main loop
    while running:
        # Calculate current song time
        if music_playing and not paused:
            current_time = (pygame.time.get_ticks() - song_start_time) / 1000
        else:
            current_time = (pygame.time.get_ticks() - song_start_time) / 1000
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_SPACE:
                    if paused:
                        pygame.mixer.music.unpause()
                        paused = False
                    else:
                        pygame.mixer.music.pause()
                        paused = True
                elif event.key == pygame.K_r:
                    # Restart song
                    pygame.mixer.music.stop()
                    pygame.mixer.music.play()
                    song_start_time = pygame.time.get_ticks()
                    paused = False
                elif event.key == pygame.K_UP:
                    volume = min(1.0, volume + 0.1)
                    pygame.mixer.music.set_volume(volume)
                elif event.key == pygame.K_DOWN:
                    volume = max(0.0, volume - 0.1)
                    pygame.mixer.music.set_volume(volume)
        
        if not paused and music_playing:
            # Update train
            train.update(current_time)
            
            # Add occasional particles
            if random.random() > 0.8:
                particles.append(
                    Particle(
                        random.randint(0, WIDTH), random.randint(0, HEIGHT),
                        (random.randint(200, 255), random.randint(200, 255), random.randint(200, 255)),
                        (random.uniform(-1, 1), random.uniform(-1, 1)),
                        random.uniform(1, 3),
                        0.95
                    )
                )
            
            # Update particles
            for particle in particles[:]:
                particle.update()
                if particle.life <= 0:
                    particles.remove(particle)
        
        # Draw everything
        draw_background()
        
        # Draw particles
        for particle in particles:
            particle.draw(screen)
        
        # Draw train
        train.draw(screen)
        
        # Draw lyrics
        draw_lyrics(current_time)
        
        # Draw progress bar
        draw_progress_bar(current_time)
        
        # Draw volume controls
        draw_volume_controls(volume)
        
        # Draw controls
        draw_controls()
        
        # Draw pause indicator
        if paused:
            pause_surface = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            pause_surface.fill((0, 0, 0, 150))
            screen.blit(pause_surface, (0, 0))
            
            pause_text = title_font.render("PAUSED", True, HIGHLIGHT_COLOR)
            screen.blit(pause_text, (WIDTH//2 - pause_text.get_width()//2, HEIGHT//2 - 50))
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
Loaded placeholder song. Replace with your actual song file.


SystemExit: 

/home/rustom/anaconda3/envs/aienv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
